<a href="https://colab.research.google.com/github/CalvHobbes/pricecomp_agents/blob/main/pydantic/pydantic_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install 'pydantic-ai[logfire]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.9/177.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.6/166.6 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.3/64.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.1/118.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.7/319.7 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib_metadata 8.6.1
    Uninstalling importlib_metadata-8.6.1:
      Successfully uninstalled importlib_metadata-8.6.1
  Attempting uninstall: opentelemetry-api
    Found existing installation: opentelemetry-api 1.16.0
    Uninstalling opentelemetry-api-1.16.0:
      Successfu

In [11]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# prompt: check if in colab environment and if so, fetch helper functions for tools

import sys

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_Croma.py
  !wget https://raw.githubusercontent.com/CalvHobbes/shop_india/main/llamaindex/search_reliance.py

In [7]:
from google.colab import userdata
import os

openapi_key = userdata.get('OPENAPI_KEY')
hf_token = userdata.get('HF_TOKEN')
os.environ["OPENAI_API_KEY"] = openapi_key

In [2]:
instructions = '''
You are an intelligent assistant tasked with analyzing and combining product price data by executing one or more tools available to you. to run the tools you must provide them the name of the product the user is searching.. Your goal is to identify the most relevant product prices and present them in a structured format.  Execute all applicable tools.Follow these steps:

Combine Results: Analyze the product data from all websites. If product names or display names are slightly different but refer to the same product, use your judgment to group them together.

Identify the Best Prices: For each product, compare the prices across all websites and order them in ascending order (lowest price first).

Output Format: Present the results in the following structured format for each product:

Product Name: The most accurate or common name for the product.

For each store, include:

Store Display Name: The name of the store.

Product URL: A clickable link to the product page.

Price: The price of the product at that store.

Ensure the store-wise details are ordered by ascending price (cheapest first).

Handle Ambiguities: If product names or details are unclear, use context to determine if they refer to the same product. If uncertain, list them separately.

Sample Result:
Product Name: Wireless Noise-Cancelling Headphones
- Store: TechWorld, Product URL: [TechWorld Link], Price: $120
- Store: GadgetHub, Product URL: [GadgetHub Link], Price: $125
- Store: ElectroShop, Product URL: [ElectroShop Link], Price: $130

'''

In [4]:
model_id = "gpt-4o-mini"

In [9]:
from pydantic_ai import Agent, RunContext
from search_Croma import fetch_prices_from_croma
from search_reliance import fetch_prices_from_reliance

price_comp__agent_with_tools = Agent(
    model_id,
    deps_type=str,
    result_type=str,
    system_prompt=(
        'Use the `search_croma_tool`  and `search_reliance_tool` functions to determine best '
        'price for a product.'
    ),
)

@price_comp__agent_with_tools.tool
def search_croma_tool(ctx: RunContext[str],product: str) -> str:
    """
    This is a tool that searches for the user specified product on Croma and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_croma(product)


@price_comp__agent_with_tools.tool
def search_reliance_tool(ctx: RunContext[str],product: str) -> str:
    """
    This is a tool that searches for the user specified product on Reliance and returns the name, url and price of the product in JSON

    Args:
        product: The product name to search for
    """
    return fetch_prices_from_reliance(product)

In [13]:
result = price_comp__agent_with_tools.run_sync("what is the best price for iphone 16 pro max")
print(result.data)

Here are the best prices for the iPhone 16 Pro Max from Croma and Reliance:

### Croma
1. **[Apple iPhone 16 Pro Max (256GB, Black Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-black-titanium-/p/309742)** - ₹137,900
2. **[Apple iPhone 16 Pro Max (256GB, Desert Titanium)](https://www.croma.com/apple-iphone-16-pro-max-256gb-desert-titanium-/p/309746)** - ₹137,900
3. **[Apple iPhone 16 Pro Max (512GB, Black Titanium)](https://www.croma.com/apple-iphone-16-pro-max-512gb-black-titanium-/p/309748)** - ₹157,900

### Reliance
1. **[Apple iPhone 16 Pro Max 256 GB, Black Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-black-titanium/p/494423059)** - ₹137,900
2. **[Apple iPhone 16 Pro Max 256 GB, White Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-256-gb-white-titanium/p/494423060)** - ₹137,900
3. **[Apple iPhone 16 Pro Max 512 GB, White Titanium](https://www.reliancedigital.in/apple-iphone-16-pro-max-512-gb-white-titanium/p/494423064)**

In [14]:
result = await price_comp__agent_with_tools.run('best price for samsung s25?')
print(result.data)



Here are the best prices for the Samsung Galaxy S25 from Croma and Reliance:

### Croma
1. **[SAMSUNG Galaxy S25 (12GB RAM, 256GB, Icyblue)](https://www.croma.com/samsung-galaxy-s25-5g-12gb-ram-256gb-icyblue-/p/313341)**
   - Price: ₹80,999

2. **[SAMSUNG Galaxy S25 (12GB RAM, 512GB, Icyblue)](https://www.croma.com/samsung-galaxy-s25-5g-12gb-ram-512gb-icyblue-/p/313335)**
   - Price: ₹92,999

### Reliance
1. **[Samsung Galaxy S25 5G 256 GB, 12 GB RAM, Icyblue](https://www.reliancedigital.in/samsung-galaxy-s25-5g-256-gb-12-gb-ram-icyblue-mobile-phone/p/494493901)**
   - Price: ₹80,999

2. **[Samsung Galaxy S25 5G 512 GB, 12 GB RAM, Icyblue](https://www.reliancedigital.in/samsung-galaxy-s25-5g-512-gb-12-gb-ram-icyblue-mobile-phone/p/494493907)**
   - Price: ₹92,999

### Best Price Summary:
- The **best price** for the **Samsung Galaxy S25 (256GB)** is **₹80,999** available at both **Croma** and **Reliance**.
- The **Samsung Galaxy S25 (512GB)** is **available for ₹92,999** at both stores

In [15]:
async with price_comp__agent_with_tools.run_stream('What is the cheapest iphone 13') as response:
    print(await response.get_data())

The cheapest iPhone 13 options are:

1. **Apple iPhone 13 (256GB, Alpine Green)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-alpine-green-/p/249841)

2. **Apple iPhone 13 (256GB, Starlight White)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-starlight-white-/p/243465)

3. **Apple iPhone 13 (256GB, Pink)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-pink-/p/243466)

4. **Apple iPhone 13 (256GB, Midnight)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-midnight-/p/243464)

5. **Apple iPhone 13 (256GB, Blue)**
   - Price: ₹43,994
   - [View Product](https://www.croma.com/apple-iphone-13-256gb-blue-/p/243468)

There are no listings for the iPhone 13 from Reliance at this time. All the listed options from Croma are for the 256GB variant, which is currently the best price available.
